## Next Song

This is a script that is part of the Next Song package depending on a certain genre. In this case the country genre.

#### Import all necessary libraries

In [1]:
import base64
import random
from urllib.parse import urlencode

import requests
import spotipy
from bs4 import BeautifulSoup
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
class Songs:

    def __init__(self):
        # initilaize the spotify credentials
        self.spoti_cid = "5d7c819c734a42849b1b5b5a00380a25"
        self.spoti_client_secret = "4f13b1d932f84c4eb5c5f7c31ee8b22b"

    # method 1 returns a new random song
    def get_random_Song(self):
        print("========================================================================\n")
        client_credentials_manager = SpotifyClientCredentials(client_id=self.spoti_cid,
                                                              client_secret=self.spoti_client_secret)
        sp_spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

        country_uri = 'spotify:playlist:37i9dQZF1DWTkxQvqMy4WW'

        country_playlist = sp_spotify.playlist(country_uri)

        country_playlist = country_playlist["tracks"]["items"]
        song_details = []
        for song in country_playlist:
            # access all items in the tracks, where each item represents a track
            song = song["track"]["album"]
            artist_name = song["artists"][0]["name"]
            song_title = song["name"]
            album_cover = song["images"][0]["url"]

            spoti_song_info = {
                "Artist name": artist_name,
                "Song title": song_title,
                "Album cover": album_cover
            }
            song_details.append(spoti_song_info)

        randNum = random.randint(0, len(song_details))

        artist_name = song_details[randNum]["Artist name"]  # title
        print("Artist: ", artist_name)

        song_title = song_details[randNum]["Song title"]  # title
        print("Song title: ", song_title)

        album_cover = song_details[randNum]["Album cover"]  # title
        print("Cover: ", album_cover, end="\n\n================="
                                          "================="
                                          "================"
                                          "======================\n")

    def highest_rated_song(self):
        req = requests.get("https://www.billboard.com/charts/country-songs/")
        soup = BeautifulSoup(req.content, 'html.parser')

        # Artist name
        artist_name = soup.find('p', {'class': 'lrv-u-margin-tb-00'})
        print("Artist: ", artist_name.text)
        # Song title
        song_title = soup.find('a', {'class': 'c-title__link lrv-a-unstyle-link'})
        print("Song title: ", song_title.text.strip())
        # Rating One
        ranking = soup.find('p', {'class': 'a-font-primary-medium-xxs'})
        print("Ranking: ", ranking.text.strip())
        # song_cover
        song_cover = soup.find('img', {'class': 'lrv-u-width-100p'})
        song_cover = song_cover['src']
        print("Song cover: ", song_cover, end="\n\n================="
                                              "================="
                                              "================"
                                              "======================\n")

    def search_song(self, keyword):
        client_creds = f"{self.spoti_cid}:{self.spoti_client_secret}"
        client_cred_b64 = base64.b64encode(client_creds.encode()).decode()
        token_url = "https://accounts.spotify.com/api/token"
        token_data = {
            "grant_type": "client_credentials"
        }
        token_headers = {
            "Authorization": f"Basic {client_cred_b64}"
        }

        # send the authorization request using POST method
        req = requests.post(token_url, data=token_data, headers=token_headers)
        token_response_data = req.json()

        access_token = token_response_data['access_token']
        expires_in = token_response_data['expires_in']  # seconds
        token_type = token_response_data['token_type']
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        endpoint = "https://api.spotify.com/v1/search"

        data = urlencode({
            "q": f"{keyword}",
            "type": "track"
        })

        lookup_url = f"{endpoint}?{data}"

        # Retrieve/ Get/ Fetch data
        searched_song_data = requests.get(lookup_url, headers=headers)
        searched_song_json = searched_song_data.json()

        searched_song_json = searched_song_json["tracks"]["items"]
        song_details = []

        # Extract particular values
        for song in searched_song_json:
            # access all items in the tracks, where each item represents a track
            artist_name = song["artists"][0]["name"]
            song_title = song["name"]
            ranking_one = song["popularity"]
            album_cover = song["album"]["images"][0]["url"]

            spoti_song_info = {
                "Artist name": artist_name,
                "Song title": song_title,
                "Ranking": ranking_one,
                "Album cover": album_cover
            }
            song_details.append(spoti_song_info)

        randNum = random.randint(0, len(song_details))

        artist_name = song_details[randNum]["Artist name"]  # title
        print("Artist: ", artist_name)

        song_title = song_details[randNum]["Song title"]  # title
        print("Song title: ", song_title)

        ranking = song_details[randNum]["Ranking"]  # title
        print("Ranking: ", ranking)

        album_cover = song_details[randNum]["Album cover"]  # title
        print("Cover: ", album_cover, end="\n\n================="
                                          "================="
                                          "================"
                                          "======================\n")

songs = Songs()
songs.get_random_Song()
songs.highest_rated_song()
keyword = input("Search: ")
songs.search_song(keyword)


Artist:  Zac Brown Band
Song title:  The Foundation
Cover:  https://i.scdn.co/image/ab67616d0000b2733f6f4ab78c05dfdfcc37a205

Artist:  Morgan Wallen
Song title:  You Proof
Ranking:  Highest ranking debut
Song cover:  https://www.billboard.com/wp-content/themes/vip/pmc-billboard-2021/assets/public/lazyload-fallback.gif

Search: the weeknd
Artist:  The Weeknd
Song title:  Sacrifice
Ranking:  81
Cover:  https://i.scdn.co/image/ab67616d0000b2734ab2520c2c77a1d66b9ee21d

